# What is NAT ? 
* internet is a big network 
* Number of devices are more than number of addresses in 32bit space 
* IP addressing is managed (public, private) : RFC1918
* private to public translation : NAT
* original NAT was 1:1 translation - during access one prvate address exclusively got translated into a public one
    - Not feasible approach in present day
    - it used to use a translation table (private : public) called a NAT table
* NAT was extende to meet requirements using Port Address Translation (PAT)
    - _overlads_ a single public address to a number of private addresses 
    - PAT table = Indside Address __(private ip + Source port) : Outside address (Public IP + Destiation port)__
    - ports are generated by the OS (Use __`netstat`__ to see all active ports)
    - even if two end devices generates same port number, collision is detected in router and asks the later PC to generate the next available port number. 

```
$ netstat | more

Active Internet connections (w/o servers)
Proto Recv-Q Send-Q Local Address           Foreign Address         State      
tcp        0      0 localhost:55812         localhost:45139         ESTABLISHED
tcp        0      0 localhost:8888          localhost:55556         ESTABLISHED
tcp        0      0 localhost:38979         localhost:40784         ESTABLISHED
tcp        0      0 localhost:40776         localhost:38979         ESTABLISHED
tcp        0      0 localhost:8888          localhost:55594         ESTABLISHED
tcp        0      0 rishi350:53652          actiontoad.canonic:http TIME_WAIT  
tcp        0      0 localhost:58194         localhost:59873         ESTABLISHED
tcp        0      0 localhost:59873         localhost:58202         ESTABLISHED
tcp        0      0 localhost:40784         localhost:38979         ESTABLISHED
tcp        0      0 localhost:34292         localhost:45901         ESTABLISHED
tcp        0      0 localhost:55594         localhost:8888          ESTABLISHED
tcp        0      0 localhost:45901         localhost:34292         ESTABLISHED
tcp        0      0 localhost:55556         localhost:8888          ESTABLISHED
tcp        0      0 localhost:58202         localhost:59873         ESTABLISHED
tcp        0      0 localhost:59873         localhost:58194         ESTABLISHED
tcp        0      0 localhost:38979         localhost:40776         ESTABLISHED
tcp        0      0 rishi350:55536          ec2-54-149-145-19:https ESTABLISHED

```

## Static NAT
* configure ports statically.
* typically static nats are configureed for cetering incoming requests (e.g. RDP) 

# NAT Overload configuration

![](nat1.png)

## Base config
```
!r1
conf t
    ! int config 
    int f0/0
        ip add dhcp 
        no sh 
    int f0/1
        ip add 10.1.2.1 255.255.255.0
        no sh 
    exit
    ! set static route 
    ip route 0.0.0.0 0.0.0.0 f0/0
    
    ! dhcp config
    ip dhcp pool test
        network 10.1.2.0 /24
        default-rauter 10.1.2.1
        dns 8.8.8.8
    exit
    ip dhcp exclude 10.1.2.1 10.1.2.100
end 
```

## Need of NAT
* Ping from R1 to 8.8.8.8 is __successful via F0/0__

```
R1#ping 8.8.8.8 source f0/0

Type escape sequence to abort.
Sending 5, 100-byte ICMP Echos to 8.8.8.8, timeout is 2 seconds:
Packet sent with a source address of 192.168.122.45 
!!!!!
Success rate is 100 percent (5/5), round-trip min/avg/max = 60/60/60 ms

```
* Ping from R1 to 8.8.8.8 is not __successful via F0/1__
        
```
R1#ping 8.8.8.8 source f0/1

Type escape sequence to abort.
Sending 5, 100-byte ICMP Echos to 8.8.8.8, timeout is 2 seconds:
Packet sent with a source address of 10.1.2.1 
.....
Success rate is 0 percent (0/5)
```
![](icmp_cap1)

### Basic facts about NAT
* WHY ?
    - see the wireshark capture
    - check there's no ICMP reply for requests from `f0/1` which is not the case of `f0/0`
    - the private IP address range `10.1.2.0/24` is routed to the ISP, whcih is blocking it using its ACL
* SOLUTION ?
    - configure NAT in R1 to translate private address space to public 
    - configure NAT overload to the Exit interface (`f0/0`)
        - step 1 : Create ACL to identify addresses to be tranlated (NAT can't be configured w/o ACl)
        - step 2 : Identify inside and outside interfaces 
        - step 3 : activate NAT
        - verify NAT alloation : `sh ip nat translation`
            - __protocol:__ which protocl was used
            - __inside local:__ port number generated for inside private address
            - __inside globla:__ port number generated for translated public address
            - __outside local:__ port number received at public destination 
            - __outside global:__ port number of the inside address of destination
      
* how to bring internet access to the hosts (ub1, ub2) ? 

## Configuration
```
conf t
    !step 1 : crete a standard ACL to identify addresses
    ip access-list statndard NAT_ADDR
        permit 10.1.2.0 0.0.0.255     !target private space
    
    !step 2 : identify inside (ingress) and outside (egreess) interface
    int f0/1
        ip nat inside  ! enabling nat process for inside interface
    int f0/0
        ip nat outside ! enabling nat process for inside interface
    exit
    
    !step 3 : define NAT operation
    ip nat inside source list NAT_ADDR interface f0/0 overload 
            ! do a inside-out NAT for source addr in ACL=NAT_ADDR 
            ! translate out to interface f0/0
            ! Overload : enable PAT (otherwise static NAT)
end
```

### Verification
__See nat translation__
```
    R1#sh ip nat translations 
    Pro Inside global      Inside local       Outside local      Outside global
    udp 192.168.122.45:38661 10.1.2.103:38661 8.8.8.8:53         8.8.8.8:53
    tcp 192.168.122.45:38964 10.1.2.103:38964 91.189.88.152:80   91.189.88.152:80
    udp 192.168.122.45:58499 10.1.2.103:58499 8.8.8.8:53         8.8.8.8:53

```

__Test Connectivity__
```
    R1#ping 8.8.8.8 source f0/1

    Type escape sequence to abort.
    Sending 5, 100-byte ICMP Echos to 8.8.8.8, timeout is 2 seconds:
    Packet sent with a source address of 10.1.2.1 
    !!!!!
    Success rate is 100 percent (5/5), round-trip min/avg/max = 60/62/68 ms
```
__Ping google.com from host__
```
    root@ub1:~# ping www.google.com

    PING www.google.com (216.58.204.4) 56(84) bytes of data.
    64 bytes from lhr48s21-in-f4.1e100.net (216.58.204.4): icmp_seq=1 ttl=53 time=20.0 ms
    64 bytes from lhr48s21-in-f4.1e100.net (216.58.204.4): icmp_seq=2 ttl=53 time=16.8 ms
    64 bytes from lhr48s21-in-f4.1e100.net (216.58.204.4): icmp_seq=3 ttl=53 time=14.6 ms
    ^C
    --- www.google.com ping statistics ---
    3 packets transmitted, 3 received, 0% packet loss, time 2002ms
    rtt min/avg/max/mdev = 14.677/17.203/20.047/2.208 ms
    root@ub1:~#
```

## NAT Pool configuration
* when you're multi-homed or having multiple public IP addess
* use a pool of public IP you have and overload them 
```
!R1
conf t
    ip nat pool [pool-name] [start-addr] [end-addr] [cidr]
    ip nat inside source list [acl-name] pool [pool-name] overload
end 
```

# Static NAT configurtion
* primarily geared for traffic coming into your networks from outside 
* normally these are blocked by the firewall 
* Adviced to use __static__ inside IP addresses 

```
! 192.168.1.50  <---- 200.1.1.10
! ip nat inside source static [iloc] [iglo]

    ip nat inside source static 192.168.1.51 200.1.1.10

! 19.168.1.50 <---- 200.1.1.100:80/443
! ip nat inside source static [tcp|udp] [iloc] [iport] [iglo] [oport]

    ip nat inside source static tcp 192.168.1.51 80 200.1.1.10 80
    ip nat inside source static tcp 192.168.1.51 443 200.1.1.10 443
```